# USAU College Nationals Data Prep

In [40]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import matplotlib.pyplot as plt

In [49]:
pool = pd.read_csv('./data/ultimate/collegenationals_men/all_poolplay.csv')
pool['pnt_diff'] = pool['home_score'] - pool['away_score']
pool['home_W'] = pool['home_score'] > pool['away_score']
pool.head()

,year,event,table,Date,Time,home_team,away_team,home_score,away_score,home_seed,away_seed,pnt_diff,home_W
0,2014,USA-Ultimate-D-I-College-Championships/schedul...,Pool A Schedule & Scores,Fri 5/23,10:30 AM,Colorado (1),Harvard (8),15,11,1,8,4,True
1,2014,USA-Ultimate-D-I-College-Championships/schedul...,Pool A Schedule & Scores,Fri 5/23,10:30 AM,Wisconsin (12),California-San Diego (13),15,7,12,13,8,True
2,2014,USA-Ultimate-D-I-College-Championships/schedul...,Pool A Schedule & Scores,Fri 5/23,2:30 PM,Harvard (8),California-San Diego (13),15,14,8,13,1,True
3,2014,USA-Ultimate-D-I-College-Championships/schedul...,Pool A Schedule & Scores,Fri 5/23,2:30 PM,Wisconsin (12),Eastern Michigan (17),15,11,12,17,4,True
4,2014,USA-Ultimate-D-I-College-Championships/schedul...,Pool A Schedule & Scores,Fri 5/23,4:30 PM,Colorado (1),Eastern Michigan (17),15,9,1,17,6,True


In [51]:
info = pool[['home_seed','away_seed','pnt_diff','home_W']].groupby(['home_seed','away_seed']).agg(['sum'])

pnt_diff home_W
                         sum    sum
home_seed away_seed                
1         8               20    5.0
          12              25    6.0
          13              19    4.0
          17              45    6.0
2         7               30    5.0
          11               8    3.0
          14              22    5.0
          18              32    6.0
3         6               12    4.0
          10              14    4.0
          15               4    4.0
          19              14    4.0
4         5                5    3.0
          9               12    5.0
          16              -3    3.0
          20              49    6.0
5         9               -2    3.0
          16              11    4.0
          20              34    6.0
6         10              16    4.0
          15              25    6.0
          19              32    6.0
7         11             -13    1.0
          14              16    4.0
          18              18    6.0
8         12               7    5.0
          13               9    4.0
          17              15    5.0
9         16              21    5.0
          20              38    6.0
10        15               0    3.0
          19              20    5.0
11        14              26    6.0
          18              20    5.0
12        13               8    4.0
          17               7    4.0
13        17              -2    3.0
14        18              12    5.0
15        19              11    4.0
16        20              17    5.0